# Canny Parameter Selection Analysis

## Overview

The canny edge detector is an algorithm that is used to identify edges with in an image. There are three primary objectives the algorithm attempts to meet.

1. Detection of edge with low error rate.
2. The edge point detected from the operator should accurately localize on the center of the edge.
3. A given edge in the image should ony be marked once.

The canny edge dector performs the following operations.

1. Apply Guassian filter to smooth the image in order to remove the noise.
2. Find the intensity gradients of the image.
3. Apply non-maximum suppression to get rid of spurious response to edge detection.
4. Apply double threshold to determine potential edges.
5. Finalize the detection of edges by suppressing all the other edges that are weak and not connected to strong edges.

### Guassian Filter

**Filters image noise.**

It is important to understand that the selection of the size of the Gaussian kernel will affect the performance of the detector. The larger the size is, the lower the detector’s sensitivity to noise. Additionally, the localization error to detect the edge will slightly increase with the increase of the Gaussian filter kernel size. A 5×5 is a good size for most cases, but this will also vary depending on specific situations.

### Finding the intensity gradient of the image

**Four filters are used to detect horizontal, vertical, and diagonal edges in the blurred image.** 



In [1]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2

%matplotlib qt5

In [2]:
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

## Image Examples

In [3]:
canny_params = dict()
canny_params['low_threshold'] = 75
canny_params['high_threshold'] = 200

In [4]:
img = mpimg.imread('test_images/solidWhiteCurve.jpg')

# convert image to grayscale
gray = grayscale(img)

# blur image
blur = gaussian_blur(gray, kernel_size=5)

# find edges
edges = canny(blur, **canny_params)

plt.subplot(2,1,1)
plt.imshow(img)

plt.subplot(2,1,2)
plt.imshow(edges, cmap="gray")
plt.show()

In [5]:
img = mpimg.imread('test_images/solidWhiteRight.jpg')

# convert image to grayscale
gray = grayscale(img)

# blur image
blur = gaussian_blur(gray, kernel_size=5)

# find edges
edges = canny(blur, **canny_params)

plt.subplot(2,1,1)
plt.imshow(img)

plt.subplot(2,1,2)
plt.imshow(edges, cmap="gray")
plt.show()

In [6]:
img = mpimg.imread('test_images/solidYellowCurve2.jpg')

# convert image to grayscale
gray = grayscale(img)

# blur image
blur = gaussian_blur(gray, kernel_size=5)

# find edges
edges = canny(blur, **canny_params)

plt.subplot(2,1,1)
plt.imshow(img)

plt.subplot(2,1,2)
plt.imshow(edges, cmap="gray")
plt.show()

In [7]:
img = mpimg.imread('test_images/solidYellowLeft.jpg')

# convert image to grayscale
gray = grayscale(img)

# blur image
blur = gaussian_blur(gray, kernel_size=5)

# find edges
edges = canny(blur, **canny_params)

plt.subplot(2,1,1)
plt.imshow(img)

plt.subplot(2,1,2)
plt.imshow(edges, cmap="gray")
plt.show()

In [8]:
img = mpimg.imread('test_images/whiteCarLaneSwitch.jpg')

# convert image to grayscale
gray = grayscale(img)

# blur image
blur = gaussian_blur(gray, kernel_size=5)

# find edges
edges = canny(blur, **canny_params)

plt.subplot(2,1,1)
plt.imshow(img)

plt.subplot(2,1,2)
plt.imshow(edges, cmap="gray")
plt.show()

In [9]:
img = mpimg.imread('test_images/solidYellowCurve.jpg')

# convert image to grayscale
gray = grayscale(img)

# blur image
blur = gaussian_blur(gray, kernel_size=5)

# find edges
edges = canny(blur, **canny_params)

plt.subplot(2,1,1)
plt.imshow(img)

plt.subplot(2,1,2)
plt.imshow(edges, cmap="gray")
plt.show()

## Trial and Error Results

It looks like a lower threshold of **75** and high threshold of **200** works best for these images.

## Region of Interest Parameter Selection

Time to find the best parameters for region selection.

In [10]:
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [365]:
# find all image files
img_dir = "test_images/"
img_files = [os.path.join(img_dir, i) for i in os.listdir(img_dir)]
for img_fp in img_files:
    img = mpimg.imread(img_fp)
    gray = grayscale(img)
    blur = gaussian_blur(gray, kernel_size=5)
    edges = canny(blur, low_threshold=75, high_threshold=200)
    
#     # center vertical line
#     cv2.line(edges, (480, 0), (480, 540), color=[255, 0, 0], thickness=5)
    
#     # horizontal line
#     cv2.line(edges, (0, 335), (960, 335), color=[255, 0, 0], thickness=5)
    
#     # left vertical line
#     cv2.line(edges, (400, 335), (60, 540), color=[255, 0, 0], thickness=5)
    
#     # right vertical line
#     cv2.line(edges, (560, 335), (960, 540), color=[255, 0, 0], thickness=5)
    
    verticies = np.array([[(400, 335), (60, 540), (960, 540), (560, 335)]])
    region = region_of_interest(edges, vertices=verticies)
    
    # plot image
    plt.figure()
    plt.imshow(region, cmap="gray")

plt.show()

## Trial and Error Results

The best region I found was a trapezoid composed of the following points:

* (400, 335) 
* (60, 540) 
* (960, 540) 
* (560, 335)

In [76]:
x_window = {'low': 60,
            'high': 960}
y_window = {'low': 335,
            'high': 540}


## Hough Transform Parameter Selection

In [265]:
def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
            
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def process_image(img):
    """
    This pipline includes grayscale, gaussian, edges, and region of interest with parameters identified above.
    """
    gray = grayscale(img)
    blur = gaussian_blur(gray, kernel_size=5)
    edges = canny(blur, low_threshold=75, high_threshold=200)
    verticies = np.array([[(400, 335), (60, 540), (960, 540), (560, 335)]])
    region = region_of_interest(edges, vertices=verticies)
    return region
    
    

In [13]:
hough_params = {'rho': 1,
                'theta': np.pi/180,
                'threshold': 15,
                'min_line_len': 15,
                'max_line_gap': 5}

In [14]:
img = mpimg.imread("test_images/solidWhiteCurve.jpg")
region = process_image(img)
lines = hough_lines(region, **hough_params)

cv2.line(lines, (480,0), (480, 540), color=[255,0,0], thickness=5)

plt.imshow(lines, cmap="gray")
plt.show()

In [15]:
img = mpimg.imread("test_images/solidWhiteRight.jpg")
region = process_image(img)
lines = hough_lines(region, **hough_params)

cv2.line(lines, (480,0), (480, 540), color=[255,0,0], thickness=5)

plt.imshow(lines, cmap="gray")
plt.show()

In [16]:
img = mpimg.imread("test_images/solidYellowCurve.jpg")
region = process_image(img)
lines = hough_lines(region, **hough_params)

cv2.line(lines, (480,0), (480, 540), color=[255,0,0], thickness=5)

plt.imshow(lines, cmap="gray")
plt.show()

In [17]:
img = mpimg.imread("test_images/solidYellowCurve2.jpg")
region = process_image(img)
lines = hough_lines(region, **hough_params)

cv2.line(lines, (480,0), (480, 540), color=[255,0,0], thickness=5)

plt.imshow(lines, cmap="gray")
plt.show()

In [18]:
img = mpimg.imread("test_images/solidYellowLeft.jpg")
region = process_image(img)
lines = hough_lines(region, **hough_params)

cv2.line(lines, (480,0), (480, 540), color=[255,0,0], thickness=5)

plt.imshow(lines, cmap="gray")
plt.show()

In [19]:
img = mpimg.imread("test_images/whiteCarLaneSwitch.jpg")
region = process_image(img)
lines = hough_lines(region, **hough_params)

cv2.line(lines, (480,0), (480, 540), color=[255,0,0], thickness=5)

plt.imshow(lines, cmap="gray")
plt.show()

## Slope Analysis Given Hough Parameters

In [20]:
img_dir = "test_images/"
img_files = [os.path.join(img_dir, i) for i in os.listdir(img_dir)]


for img_fp in img_files:
    slopes = []
    lengths = []
    
    img = mpimg.imread(img_fp)
    region = process_image(img)
    lines = cv2.HoughLinesP(region, 
                                   hough_params['rho'], 
                                   hough_params['theta'], 
                                   hough_params['threshold'], 
                                   np.array([]), 
                                   minLineLength=hough_params['min_line_len'], 
                                   maxLineGap=hough_params['max_line_gap'])

    for line in lines:
        for x1, y1, x2, y2 in line:
            slope = (y2-y1)/(x2-x1)
            slopes.append(slope)
            
            length = np.sqrt((x2-x1)**2 + (y2-y1)**2)
            lengths.append(length)
    
    plt.figure()
    plt.subplot(2,1,1)
    plt.imshow(region, cmap="gray")
    plt.subplot(2,1,2)
    plt.plot(slopes, lengths, 'x')
plt.show()

## Movie Frame Analysis

Need to make sure parameter selection is robust for all images in video

In [21]:
def process_image_final(img):
    region = process_image(img)
    lines = cv2.HoughLinesP(region, 
                            hough_params['rho'], 
                            hough_params['theta'], 
                            hough_params['threshold'], 
                            np.array([]), 
                            minLineLength=hough_params['min_line_len'], 
                            maxLineGap=hough_params['max_line_gap'])
    for line in lines:
        for x1, y1, x2, y2 in line:
            dx = (x2 - x1)
            dy = (y2 - y1)
            slope = dy/dx
            length = np.sqrt((dy)**2 + (dx)**2)
            centroid = dx/2 + x1, dy/2 + y1
            slopes.append(slope)
            lengths.append(length)
            centroids.append(centroid)    
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

In [22]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [23]:
# reset data
slopes = []
lengths = []
centroids = []

white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image_final) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

slopes_white = np.copy(np.array(slopes))
lengths_white = np.copy(np.array(lengths))
centroids_white = np.copy(np.array(centroids))

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|█████████▉| 221/222 [00:05<00:00, 38.87it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

CPU times: user 14.9 s, sys: 516 ms, total: 15.4 s
Wall time: 5.87 s


In [24]:
plt.subplot(2,1,1)
plt.plot(slopes_white, lengths_white, 'x')
plt.subplot(2,1,2)
plt.plot(centroids_white[:,0], centroids_white[:,1], 'x')
plt.gca().invert_yaxis()
plt.show()

In [25]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [26]:
# reset data
slopes = []
lengths = []
centroids = []

yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image_final)
%time yellow_clip.write_videofile(yellow_output, audio=False)

slopes_yellow = np.copy(np.array(slopes))
lengths_yellow = np.copy(np.array(lengths))
centroids_yellow = np.copy(np.array(centroids))

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|█████████▉| 681/682 [00:17<00:00, 38.78it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

CPU times: user 46.2 s, sys: 1.6 s, total: 47.8 s
Wall time: 17.4 s


In [27]:
plt.subplot(2,1,1)
plt.plot(slopes_yellow, lengths_yellow, 'x')
plt.subplot(2,1,2)
plt.plot(centroids_yellow[:,0], centroids_yellow[:,1], 'x')
plt.gca().invert_yaxis()
plt.show()

In [28]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

In [316]:
plt.subplot(2,1,1)
plt.plot(slopes_white, lengths_white, 'bx', slopes_yellow, lengths_yellow, 'ro')
plt.subplot(2,1,2)
plt.plot(centroids_white[:,0], centroids_white[:,1], 'bx', centroids_yellow[:,0], centroids_yellow[:,1], 'ro')
plt.gca().invert_yaxis()
plt.show()

In [317]:
# Data Analysis

# concatenate and white and yellow data
slopes = np.concatenate((slopes_white, slopes_yellow))
lengths = np.concatenate((lengths_white, lengths_yellow))
centroids = np.concatenate((centroids_white, centroids_yellow))

# plot slopes and centroids
plt.figure()
plt.subplot(2,1,1)
plt.plot(slopes, lengths, 'x')
plt.subplot(2,1,2)
plt.plot(centroids[:,0], centroids[:,1], 'x')

# split data between left and right side of image
left, = np.where(centroids[:,0] < 480)
right, = np.where(centroids[:,0] > 480)

# slice left side of data
sl = slopes[left]
ll = lengths[left]
cl = centroids[left, :]

plt.figure()
plt.subplot(2,1,1)
plt.plot(sl, ll, 'x')
plt.subplot(2,1,2)
plt.plot(cl[:,0], cl[:,1], 'x')

# slice right side of data
sr = slopes[right]
lr = lengths[right]
cr = centroids[right, :]

plt.figure()
plt.subplot(2,1,1)
plt.plot(sr, lr, 'x')
plt.subplot(2,1,2)
plt.plot(cr[:,0], cr[:,1], 'x')

plt.show()

In [318]:
x_intercept = lambda y, slope, y_intercept: (y - y_intercept)/slope

# left fit
slope_left, intercept_left = np.polyfit(cl[:,0], cl[:,1], deg=1)
xl_0 = x_intercept(y_window['low'], slope_left, intercept_left)
xl_f = x_intercept(y_window['high'], slope_left, intercept_left)

xl = np.linspace(xl_0, xl_f)
left_line = np.poly1d((slope_left, intercept_left))

# fit line to data
plt.figure()
plt.plot(cl[:,0], cl[:,1], 'x', xl, left_line(xl))

# right fit
slope_right, intercept_right = np.polyfit(cr[:,0], cr[:,1], deg=1)
xr_0 = x_intercept(y_window['low'], slope_right, intercept_right)
xr_f = x_intercept(y_window['high'], slope_right, intercept_right)

xr = np.linspace(xr_0, xr_f)
right_line = np.poly1d((slope_right, intercept_right))

# fit line to data
plt.figure()
plt.plot(cr[:,0], cr[:,1], 'x', xr, right_line(xr))

plt.show()

In [319]:
def distance_from_line(x, y, line, lower, upper):
    """
    Compute delta x of point from line
    """
    slope, intercept = line.coeffs
    x_line = x_intercept(y, slope, intercept)
    return x - x_line

def centroids_in_bounds(data, lower_bound, upper_bound):
    """
    Find centroids with y values in bound.
    """
    lower = np.where(data[:,1] > lower_bound)
    upper = np.where(data[:,1] < upper_bound)
    in_bounds = np.intersect1d(lower, upper)
    return data[in_bounds,:]
    

## Sectional Guassian Fit to Remove Outliers

In [320]:
plt.subplot(2,1,1)

y_bin_size = 10
y_bins = int(y_window['high']/y_bin_size - y_window['low']/y_bin_size)

bounds = [[v, v + y_bin_size] for v in range(y_window['low'], y_window['high'], y_bin_size)]

left_lookup = dict()
for lower_bound, upper_bound in bounds:
    key = (lower_bound, upper_bound)
    points = centroids_in_bounds(cl, lower_bound=lower_bound, upper_bound=upper_bound)
    distances = np.array([distance_from_line(p[0], p[1], left_line, lower_bound, upper_bound) for p in points])
    left_lookup[key] = (np.mean(distances), np.std(distances))
    
yl = left_line(xl)
xl_err = np.zeros_like(xl) 
for lower_bound, upper_bound in left_lookup:
    for idx, y_value in enumerate(yl):
        if lower_bound <= y_value <= upper_bound:
            std = left_lookup[(lower_bound, upper_bound)][1]
            xl_err[idx] = 2*std

plt.plot(cl[:,0], cl[:,1], 'x')
plt.errorbar(xl, yl, xerr=xl_err)
    
right_lookup = dict()
for lower_bound, upper_bound in bounds:
    key = (lower_bound, upper_bound)
    points = centroids_in_bounds(cr, lower_bound=lower_bound, upper_bound=upper_bound)
    distances = np.array([distance_from_line(p[0], p[1], right_line, lower_bound, upper_bound) for p in points])
    right_lookup[key] = (np.mean(distances), np.std(distances))
    
yr = right_line(xr)
xr_err = np.zeros_like(xr) 
for lower_bound, upper_bound in right_lookup:
    for idx, y_value in enumerate(yr):
        if lower_bound <= y_value <= upper_bound:
            std = right_lookup[(lower_bound, upper_bound)][1]
            xr_err[idx] = 2*std

plt.plot(cr[:,0], cr[:,1], 'x')
plt.errorbar(xr, yr, xerr=xr_err)
plt.show()

In [321]:
plt.subplot(2,1,2)

not_outliers = []
for lower_bound, upper_bound in bounds:
    for idx, point in enumerate(cl):
        x, y = point
        if lower_bound <= y <= upper_bound:
            d = distance_from_line(x=x, y=y, line=left_line, lower=lower_bound, upper=upper_bound)
            mn, std = left_lookup[(lower_bound, upper_bound)]
            if np.abs(d) <= 2*std:
                not_outliers.append(idx)        

cl_filter = cl[not_outliers, :]

plt.plot(cl_filter[:,0], cl_filter[:,1], 'x')
plt.errorbar(xl, yl, xerr=xl_err)
plt.show()

not_outliers = []
for lower_bound, upper_bound in bounds:
    for idx, point in enumerate(cr):
        x, y = point
        if lower_bound <= y <= upper_bound:
            d = distance_from_line(x=x, y=y, line=right_line, lower=lower_bound, upper=upper_bound)
            mn, std = left_lookup[(lower_bound, upper_bound)]
            if np.abs(d) <= 2*std:
                not_outliers.append(idx)        

cr_filter = cr[not_outliers, :]

plt.plot(cr_filter[:,0], cr_filter[:,1], 'x')
plt.errorbar(xr, yr, xerr=xr_err)
plt.show()

In [322]:
# left lane parameters
print("Left Lane Parameters")
for bound, parameters in left_lookup.items():
    print(bound, parameters)

# right lane parameters
print("Right Lane Parameters")
for bound, parameters in right_lookup.items():
    print(bound, parameters)

Left Lane Parameters
(445, 455) (1.7654389019814785, 13.014251636491556)
(495, 505) (4.0217576521538563, 19.640978577514073)
(345, 355) (-2.7527676764638689, 6.6038430374854373)
(415, 425) (0.86863740324643979, 11.169631440690816)
(435, 445) (2.0886860897085358, 11.586491048509149)
(525, 535) (-2.8862164087219582, 24.378635632537172)
(375, 385) (-2.4999758136848, 7.7889927537929982)
(335, 345) (-8.7269854282211377, 14.352782040130313)
(425, 435) (1.5492591952669006, 11.528540066674818)
(485, 495) (4.4027231641529658, 15.996877302256351)
(535, 545) (28.178816769922729, 31.376444133086018)
(465, 475) (2.2967052537130854, 14.464701112871017)
(405, 415) (-0.32155095668984562, 10.539267788287573)
(475, 485) (3.9930917279096896, 16.104771661947805)
(505, 515) (2.1787758492800546, 21.293251587720842)
(515, 525) (2.4781165767449576, 20.937012569534105)
(395, 405) (-1.3649112029282928, 9.3938820264493437)
(355, 365) (-2.5930825937629813, 6.910713167969611)
(385, 395) (-1.7064953706989565, 8.225

In [357]:
def draw_lines(img, lines, color=[255, 0, 0], thickness=10, show_originals=False):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    y_bin_size = 10
    
    l_lines = []
    r_lines = []
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            # compute slope
            dx = x2 - x1
            dy = y2 - y1
            slope = dy/dx
            
            # compute center
            center = x1 + dx/2, y1 + dy/2
            
            # left half of image
            if center[0] <= 480:
                if slope < -0.5:
                    for lower_bound, upper_bound in left_lookup:
                        if lower_bound <= center[1] < upper_bound:
                            mn, std = left_lookup[(lower_bound, upper_bound)]
                            d = distance_from_line(center[0], center[1], left_line, lower_bound, upper_bound)
                            if np.abs(d) <= 2*std:
                                if show_originals:
                                    cv2.line(img, (x1, y1), (x2, y2), color, thickness)
                                l_lines.append(np.polyfit([x1, x2], [y1, y2], deg=1))
            
            # right half of image
            else:
                if slope > 0.4:
                    for lower_bound, upper_bound in right_lookup:
                        if lower_bound <= center[1] < upper_bound:
                            mn, std = right_lookup[(lower_bound, upper_bound)]
                            d = distance_from_line(center[0], center[1], right_line, lower_bound, upper_bound)
                            if np.abs(d) <= 2*std:
                                if show_originals:
                                    cv2.line(img, (x1, y1), (x2, y2), color, thickness)
                                r_lines.append(np.polyfit([x1, x2], [y1, y2], deg=1))
    
    ll_params = np.array(l_lines)
    left_slope, left_intercept = np.mean(ll_params[:,0]), np.mean(ll_params[:,1])
    
    x0, y0 = int(x_intercept(y_window['low'], left_slope, left_intercept)), int(y_window['low'])
    xf, yf = int(x_intercept(y_window['high'], left_slope, left_intercept)), int(y_window['high'])
    
    cv2.line(img, (x0, y0), (xf, yf), color, thickness)
    
    rl_params = np.array(r_lines)
    right_slope, right_intercept = np.mean(rl_params[:,0]), np.mean(rl_params[:,1])
    
    x0, y0 = int(x_intercept(y_window['low'], right_slope, right_intercept)), int(y_window['low'])
    xf, yf = int(x_intercept(y_window['high'], right_slope, right_intercept)), int(y_window['high'])
    
    cv2.line(img, (x0, y0), (xf, yf), color, thickness)


In [358]:
def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def hough_filter(img):
    region = process_image(img)
    lines = hough_lines(region, **hough_params)
    return lines

def overlay_lines(img):
    lines = hough_filter(img)
    return weighted_img(img, lines)


In [359]:
img = mpimg.imread("test_images/whiteCarLaneSwitch.jpg")
region = process_image(img)
lines = hough_lines(region, **hough_params)

plt.imshow(lines)
plt.show()

In [360]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(overlay_lines) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|█████████▉| 221/222 [00:11<00:00, 19.28it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

CPU times: user 30.8 s, sys: 860 ms, total: 31.7 s
Wall time: 12.2 s


In [361]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [363]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(overlay_lines)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|█████████▉| 681/682 [00:45<00:00, 15.10it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

CPU times: user 2min 2s, sys: 3.43 s, total: 2min 6s
Wall time: 46.1 s


In [364]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

In [368]:
import pickle
with open('left_lane_parameters.pickle', 'wb') as f:
    pickle.dump(left_lookup, f)
with open('right_lane_parameters.pickle', 'wb') as f:
    pickle.dump(right_lookup, f)

In [373]:
left_line.coeffs

array([  -0.69169104,  643.3071443 ])